In [105]:
import random
import math
import torch
import sys
import copy
import pickle
import torch.nn as nn
import scipy as sp
import scipy.stats as sps
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [106]:
# ======================================
# Prepare Data
def make_batch(data, batch_size, window_size, shuffle=True):
    window_list = []
    for i in range(len(data) - window_size - 1):
        window = data[i: i + window_size]
        window_list.append(window)

    if shuffle:
        random.shuffle(window_list)

    n_batch = math.ceil(len(window_list) / batch_size)
    batch_list = []
    for i in range(n_batch):
        batch = window_list[i*batch_size: (i+1)*batch_size]
        batch_list.append(batch)
    batch_list = np.array(batch_list)

    return batch_list
#data = pd.read_csv("./all_metric_sort_err.csv", sep=',', index_col=False)
data = pd.read_csv("./all_metric_sort_err.csv", sep=',', index_col=False)
data = data.dropna(axis=0)
p = data.isnull().values.any()

headers = [h for h in data.keys() if not h.startswith('container_') and not h.startswith("go_") and not h.startswith("node_filesystem_") and h in forhead.keys()]
data = data[headers]
print(p)

False


In [107]:
data.head()

,node_boot_time_seconds,node_boot_time_seconds.1,node_context_switches_total,node_context_switches_total.1,node_cpu_frequency_max_hertz,node_cpu_frequency_max_hertz.1,node_cpu_frequency_max_hertz.2,node_cpu_frequency_max_hertz.3,node_cpu_frequency_max_hertz.4,node_cpu_frequency_max_hertz.5,...,node_softnet_processed_total.16,node_softnet_processed_total.17,node_softnet_processed_total.18,node_softnet_processed_total.19,node_softnet_processed_total.20,node_softnet_processed_total.21,node_softnet_processed_total.22,node_softnet_processed_total.23,node_softnet_processed_total.24,node_softnet_processed_total.25
0,1596767999,1596781334,5758230699,37974415433,2100000000,2100000000,2100000000,2100000000,2100000000,2100000000,...,25544702,23436296,24701006,46671236,47947309,44975004,46843927,45760864,63930543,44422656
1,1596767999,1596781334,5758245446,37974538311,2100000000,2100000000,2100000000,2100000000,2100000000,2100000000,...,25544717,23436551,24701007,46671276,47947361,44975035,46844112,45760923,63930635,44422703
2,1596767999,1596781334,5758264904,37974658948,2100000000,2100000000,2100000000,2100000000,2100000000,2100000000,...,25544782,23436557,24701036,46671443,47947485,44975165,46844366,45761027,63930885,44422959
3,1596767999,1596781334,5758284370,37974786059,2100000000,2100000000,2100000000,2100000000,2100000000,2100000000,...,25544782,23436557,24701036,46671450,47947498,44975181,46844387,45761054,63930967,44422966
4,1596767999,1596781334,5758305482,37974907235,2100000000,2100000000,2100000000,2100000000,2100000000,2100000000,...,25544858,23436564,24701043,46671924,47947955,44975292,46844543,45761183,63931182,44423190


In [108]:
data = data.to_numpy()
#scaler = MinMaxScaler(feature_range=(0,10))
scaler = StandardScaler()
data = scaler.fit_transform(data)

In [109]:
# ======================================
# Modeling
class SequenceModel(nn.Module):
    def __init__(self, input_size=1796, output_dim=1796, hidden_size=256, num_layers=1):
        super(SequenceModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True)
        self.scaler_bias = nn.Parameter(torch.ones(input_size, requires_grad=True))
        self.scaler = nn.Parameter(torch.ones(input_size, requires_grad=True))
        self.linear = nn.Linear(hidden_size, output_dim)

    def forward(self, x):
        x = (x + self.scaler_bias) * self.scaler
        zs, hidden = self.lstm(x)
        z = zs[:, -1]
        v = self.linear(zs)
        return v, z

In [110]:
# ======================================
# Training
window_size = 10
batch_size = 192
hidden_size = 128
use_cuda = True
n_feature = data.shape[1]

model = SequenceModel(input_size=n_feature,
                      output_dim=n_feature,
                      hidden_size=hidden_size,
                      num_layers=1)

if use_cuda:
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

n_epoch = 30000
ema_loss = None
alpha = 0.1
verbose_interval = 50

In [111]:
for epoch_i in range(n_epoch):

    batch_list = make_batch(data, batch_size, window_size+1)
    for batch_i, batch in enumerate(batch_list):
        optimizer.zero_grad()

        batch = np.array(batch)
        batch_input = batch[:, :-1, :]
        batch_output = batch[:, 1:, :]

        batch_input = torch.tensor(batch_input, dtype=torch.float32)
        batch_output = torch.tensor(batch_output, dtype=torch.float32)

        if use_cuda:
            batch_input = batch_input.cuda()
            batch_output = batch_output.cuda()
        
        v, _ = model(batch_input)

        loss = loss_fn(v, batch_output)

        loss.backward()
        optimizer.step()

        if ema_loss is None:
            ema_loss = loss.item()
        ema_loss = loss.item() * alpha + (1.-alpha) * ema_loss

    if epoch_i % verbose_interval == 0:
        print(f"{epoch_i}th epoch, loss: {ema_loss}")

0th epoch, loss: 0.3837843775749206
50th epoch, loss: 0.08462249602843495
100th epoch, loss: 0.06674098407573809
150th epoch, loss: 0.059712057492054646
200th epoch, loss: 0.055790387245457415
250th epoch, loss: 0.05216448614928952
300th epoch, loss: 0.04888100172160663
350th epoch, loss: 0.04598041778995166
400th epoch, loss: 0.04342616852078391
450th epoch, loss: 0.04157690207308597
500th epoch, loss: 0.03927909474817022
550th epoch, loss: 0.037651970576966884
600th epoch, loss: 0.03547101883491353
650th epoch, loss: 0.03387561708515976
700th epoch, loss: 0.032414921212774624
750th epoch, loss: 0.030822242195357365
800th epoch, loss: 0.029610076468130943
850th epoch, loss: 0.028681178376936647
900th epoch, loss: 0.027962448740616055
950th epoch, loss: 0.02713092194448657
1000th epoch, loss: 0.02629550804656317
1050th epoch, loss: 0.025684502575872594
1100th epoch, loss: 0.025037591485649345
1150th epoch, loss: 0.0245267129750576
1200th epoch, loss: 0.023828865255412134
1250th epoch, 

In [112]:
# ======================================
# Inference
model.eval()
model.cpu()


# Prepare train data distribution
Z = []
reconstruction_error = []

v_diff_list = []

batch_list = make_batch(data, batch_size, window_size, False)


for batch_i, batch in enumerate(batch_list):
    batch = np.array(batch)
    batch_input = batch

    batch_input = torch.tensor(batch_input, dtype=torch.float32)
    batch_output = torch.tensor(batch_output, dtype=torch.float32)

    v, z = model(batch_input)

    Z.extend(z.tolist())
    
    v  # batch_size x widnow_size x n_feature
    n_feature = v.shape[2]

    v_pred = v[:, :-1, :]  # batch_size x (window_size-1) x n_feature
    
    v_gt = batch_input[:, 1:, :]
    
    v_diff = torch.abs(v_gt - v_pred) # batch_size x (window_size-1) x n_feature
    
    v_diff = v_diff.mean(dim=1) # batch_size  x n_feature
    
    v_diff = v_diff.view(-1, n_feature)  # batch_size x n_feature  => n_sample x n_feature
    
    v_diff_list.append(v_diff.detach().cpu().numpy())
    
    # v = v.view(-1, n_feature) # batch_size  x n_feature
    batch_input.view(-1, n_feature)   
    
    # reconstruction_error.extend(torch.sum(torch.abs(v-batch_input), dim=[1,2]).detach().tolist())

Z = np.array(Z)
#reconstruction_error = np.array(reconstruction_error)

/home/ubuntu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [113]:
v_diff_mat = np.concatenate(v_diff_list)

In [114]:
v_diff_mat.shape  # n_sample x n_feature

(264, 1578)

In [115]:
n_sample = len(v_diff_mat)

In [116]:
mean_vector = v_diff_mat.mean(axis=0)  # 

In [117]:
covariance =[]

In [118]:
covariance= (1./n_sample)*np.dot((v_diff_mat-mean_vector).transpose(), v_diff_mat-mean_vector)  # [n_feature x n_sample] dot [n_sample x n_feature]  => n_featrue x n_feature  

In [119]:
covariance = covariance + np.eye(*covariance.shape) # * 1e-4

In [120]:
np.all(np.linalg.eigvals(covariance)>0)

True

In [121]:
v_diff_mat.shape

(264, 1578)

In [122]:
covariance_inv = np.linalg.inv(covariance)

In [123]:
covariance_det = np.linalg.det(covariance)

In [124]:
np.linalg.det(np.eye(*covariance.shape) )

1.0

In [125]:
covariance_inv

array([[ 1.00000000e+00,  1.27125184e-13,  1.71413083e-10, ...,
        -8.64922886e-09, -5.83728453e-09, -7.09902346e-09],
       [ 1.27125184e-13,  1.00000000e+00, -1.54718198e-10, ...,
         2.52261609e-09,  4.05711132e-10,  7.50924694e-10],
       [ 1.71413083e-10, -1.54718198e-10,  9.99994977e-01, ...,
         2.01535553e-06, -2.57900452e-06, -6.95310206e-06],
       ...,
       [-8.64922886e-09,  2.52261609e-09,  2.01535553e-06, ...,
         9.99824557e-01, -1.01223949e-04, -1.11868653e-04],
       [-5.83728453e-09,  4.05711132e-10, -2.57900452e-06, ...,
        -1.01223949e-04,  9.99859407e-01, -1.46720440e-04],
       [-7.09902346e-09,  7.50924694e-10, -6.95310206e-06, ...,
        -1.11868653e-04, -1.46720440e-04,  9.99776130e-01]])

In [126]:
with open("ckpt/multi_pdf.pickle", "wb") as f:
    pickle.dump({"mean_vector": mean_vector,
                 "covariance": covariance, 
                 "covariance_inv": covariance_inv,
                 "covariance_det": covariance_det}, f)

In [127]:
# Model Store
torch.save(model.state_dict(), "ckpt/model.ckpt")

with open("./ckpt/Z.pickle", "wb") as f:
    pickle.dump(Z, f)

In [128]:
with open("./ckpt/scaler.pickle", "wb") as f:
    pickle.dump(scaler, f)

In [129]:
 with open('ckpt/multi_pdf.pickle', 'rb') as file:
         data = pickle.load(file)

In [130]:
data

{'mean_vector': array([3.7111859e-06, 1.5762308e-06, 7.4348939e-03, ..., 2.2448460e-02,
        2.1607760e-02, 2.5424536e-02], dtype=float32),
 'covariance': array([[ 1.00000000e+00, -1.38174113e-13, -1.35951805e-10, ...,
          9.45718082e-09,  6.57555832e-09,  7.99193778e-09],
        [-1.38174113e-13,  1.00000000e+00,  1.51546428e-10, ...,
         -2.62211697e-09, -4.79782214e-10, -8.41478831e-10],
        [-1.35951805e-10,  1.51546428e-10,  1.00000507e+00, ...,
         -1.70275621e-06,  2.98215059e-06,  7.51856987e-06],
        ...,
        [ 9.45718082e-09, -2.62211697e-09, -1.70275621e-06, ...,
          1.00018470e+00,  1.09684210e-04,  1.22099183e-04],
        [ 6.57555832e-09, -4.79782214e-10,  2.98215059e-06, ...,
          1.09684210e-04,  1.00015119e+00,  1.60026291e-04],
        [ 7.99193778e-09, -8.41478831e-10,  7.51856987e-06, ...,
          1.22099183e-04,  1.60026291e-04,  1.00024093e+00]]),
 'covariance_inv': array([[ 1.00000000e+00,  1.27125184e-13,  1.71413083